In [2]:
import pandas as pd
from sqlalchemy import create_engine
import json
from utils import Logger
logger = Logger()

import boto3
from botocore.session import Session

In [ ]:
def get_rds_secret(config:dict)->str:
    secret_name = config.get("SECRET_NAME")
    region_name = config.get("REGION")
    
    session = Session(
        aws_access_key_id=config.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=config.get("AWS_SECRET_KEY"),
        region_name=region_name
    )
    
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    
    try:
        logger.log(f"retrieving secret from rds", name=__name__)
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except Exception as e:
        logger.log(f"Exception occurred while retrieving secret from rds: {e}", flag=1, name=__name__)
        raise e
    secret = get_secret_value_response['SecretString']
    logger.log(f"secret:{secret}",flag=3,name=__name__)
    return secret

In [ ]:
config_path='.aws/credentials.json'
with open(config_path, 'r') as f:
    credentials = json.load(f)
user = credentials.get("USER")
password = credentials.get("AWS_SECRET_KEY")
host = credentials.get("ENDPOINT")
port = credentials.get("PORT")
database = credentials.get("DBNAME")
connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
query = "CREATE DATABASE streamlit"
try:
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
except Exception as e:
    print(f"Exception occurred : {e}")